# Região de Confiança

A busca linear é apenas uma maneira de controlar o passo do algoritmo. Uma outra estratégia bastante comum é a **região de confiança**.
Lembre que nos métodos Quase-Newton fazemos
$$ B_k d^k = -\nabla f(x^k), $$
e que isso é equivalente a encontra o minimizador da função
$$ m_k(d) = c_k + \nabla f(x^k)^Td + \frac{1}{2} d^TB_kd. $$
A partir desse ponto reduzimos o tamanho de $\lambda_k$ até que o passo $\lambda_kd^k$ satisfaça algumas condições suficientes para garantir convergência global.

No caso de $B_k = I$, que gera o método do Gradiente, ou $B_k = \nabla^2 f(x^k)$,
nós vimos que a busca exata e a condição de Armijo funcionam suficientemente bem,
e no caso de BFGS, nós usamos a condição de Wolfe, porque queríamos que a matriz
fosse definida positiva, e Wolfe garantia isso.

Vimos, no entanto, que pode não ser fácil de achar um passo para Wolfe, pois a
direção pode não estar boa o suficiente.
Mas nesse caso, por que usar a direção? A estratégia de região de confiança nos
diz que, se a direção não está boa, é melhor buscar outra.

A ideia é muito simples. Se nosso objetivo é minimizar $m_k(d)$ para achar a direção,
mas ela pode ser muito grande, vamos limitar o seu tamanho adicionando a restrição
$\Vert d\Vert \leq \Delta_k$. Então nosso problema passa a ser
$$ \min m_k(d) \qquad \mbox{suj. a} \qquad \Vert d\Vert \leq \Delta_k. $$
$\Delta_k$ um valor positivo chamado de raio da região de confiança.
Caso o passo encontrado satisfaça uma condição de decréscimo, nós o aceitamos.
Caso contrário, nós reduzimos o raio e procuramos outra direção.

A condição de decréscimo é dada pelos valores $\mbox{Ared} = f(x^k) - f(x^k+d^k)$
e $\mbox{Pred} = m_k(0) - m_k(d^k)$, ditos *redução real* e *redução prevista*.
Pred é o quanto o modelo diminui, e o Ared é o quanto a função $f$ diminuiu.
Quanto menor for o valor de $f(x^k+d^k)$, maior será Ared. E nós escalamos
esse valor pela redução de $m_k$.
Para que o passo seja bem sucedido (e o modelo seja considerado bom o suficiente),
queremos que a redução de $f$ seja ao menos uma certa fração da redução de $m_k$.
Note que Pred é sempre positivo (se $x^k$ não é estacionário), e espera-se que
Ared também seja.
Nossa condição pode ser escrita como
$$ \rho_k = \frac{\mbox{Ared}}{\mbox{Pred}} \geq \eta_1, $$
para $\eta_1 \in [0,\frac{1}{4})$, normalmente.
Se $\rho_k \geq \eta_1$, então $x^{k+1} = x^k + d^k$, e se $\rho_k < \eta_1$,
então $x^{k+1} = x^k$ e $\Delta_{k+1} = \sigma_1\Delta_k$, onde $\sigma_1 \in (0,1)$.
Além disso, se o decréscimo for muito bom, mas a região limitou nossa movimentação,
podemos aumentá-la, i.e., se $\rho_k > \eta_2$ e $\Vert d^k\Vert = \Delta_k$, então
$\Delta_{k+1} = \sigma_2\Delta_k$, com $\eta_1 < \eta_2 < 1$ e $\sigma_2 > 1$.
Caso $\eta_1 < \rho_k < \eta_2$, então $\Delta_{k+1} = \Delta_k$.

O método está definido, mas ainda precisamos encontrar $d^k$.
Existem três estratégias principais para isso. São:

  - Moré-Sorensen
  - Dogleg
  - Steihaug

Vamos apresentar a primeira.


## Moré-Sorensen

A primeira estratégia foi criada por Moré e Sorensen [1], e consiste na afirmação de que $\bar{d}$ é a solução de
$$ \min m(d) = c + g^Td + \frac{1}{2}d^TBd \qquad \mbox{suj. a} \qquad
\Vert d\Vert \leq \Delta$$
se e somente $\bar{d}$ é factível e existe $\lambda \geq 0$ tal que
\begin{align}
(B + \lambda I)\bar{d} & = -g, \\
\lambda (\Delta - \Vert\bar{d}\Vert) & = 0, \\
(B + \lambda I) & \qquad \mbox{é definida positiva}.
\end{align}

O algoritmo consiste então de procurar $\lambda$ e $\bar{d}$ satisfazendo essas condição.
Pela segunda condição, ou $\lambda = 0$, ou $\Vert d\Vert = \Delta$.
Então, ou o passo está no interior da região, ou devemos achar um $\lambda$ para que o
passo fique exatamente sobre a região.

In [3]:
f(x) = (1 - x[1])^2 + 100*(x[2]-x[1]^2)^2
∇f(x) = [-2*(1-x[1]) - 400*x[1]*(x[2]-x[1]^2); 200*(x[2]-x[1]^2)]
B = [1000 -20; -20 2]; # Essa Hessiana foi inventada
x = [1.2;-0.8];
Δ = 10.0

10.0

In [4]:
g = ∇f(x)
d = -B\g
dot(d, g) # < 0 => De descida

-114839.42420000001

In [5]:
norm(d) < Δ

false

O passo não está estritamente dentro da região. Então buscamos $\lambda$. Chutando algum valor.

In [6]:
λ = 10.0
d = -(B+λ*eye(2))\ g;
norm(d) < Δ

false

Esse valor não foi suficiente. Como a direção ainda não está dentro da região, e temos a matriz $(B+\lambda I)^{-1}$ pesando
na direção $d$, se aumentarmos $\lambda$, devemos diminuir $d$.

In [7]:
norm(d)

36.77355594359027

In [8]:
λ = 100
d = -(B+λ*eye(2))\g;
norm(d)

4.310705795626762

In [9]:
norm(d) < Δ

true

Agora o passo está dentro. Mas queremos sobre a borda da região, isto é, $\Vert d\Vert = \Delta$. Vamos diminuir $\lambda$.

In [10]:
λ = 50
d = -(B+λ*eye(2))\g;
norm(d)

8.32728449641729

Agora que você já entendeu o processo, vou sair chutando valores.

In [11]:
λ = 30
d = -(B+λ*eye(2))\g;
norm(d)

13.533908555866484

In [12]:
λ = 40
d = -(B+λ*eye(2))\g;
norm(d)

10.302244977244742

In [13]:
λ = 45
d = -(B+λ*eye(2))\g;
norm(d)

9.20826154617898

In [14]:
λ = 42.5
d = -(B+λ*eye(2))\g;
norm(d)

9.724096693352282

In [15]:
λ = 41
d = -(B+λ*eye(2))\g;
norm(d)

10.062805621460463

In [16]:
λ = 41.5
d = -(B+λ*eye(2))\g;
norm(d)

9.94727090376149

In [17]:
λ = 41.2
d = -(B+λ*eye(2))\g;
norm(d)

10.016266328558187

E assim por diante. O valor mais próximo até agora foi $\lambda = 41.2$, e uma
estratégia para achar $\lambda$ é fazer a busca em bissecção.

Um primeiro código:

In [18]:
function more_sorensen(B, g, Δ; tol = 1e-5)
    d = -B\g
    Id = eye(length(g))
    # Verificando se λ = 0.
    if norm(d) < Δ
        return d, 0.0
    end
    
    # Agora precisamos achar um limitante superior para λ
    λhigh = 1.0
    λlow = 0.0
    d = -(B+λhigh*Id)\g
    while norm(d) > Δ
        λlow = λhigh # Se não é limitante superior, é inferior.
        λhigh *= 10.0
        d = -(B+λhigh*Id)\g
    end
    # Agora a bissecção
    λ = (λlow + λhigh)/2
    d = -(B+λ*Id)\g
    while abs(norm(d) - Δ) > tol
        if norm(d) > Δ
            λlow = λ
        else
            λhigh = λ
        end
        λ = (λlow + λhigh)/2
        d = -(B+λ*Id)\g
    end
    return d, λ
end

more_sorensen (generic function with 1 method)

In [19]:
d, λ = more_sorensen(B, g, Δ)

([-0.841577,9.96452],41.27035140991211)

Note que $\lambda$ realmente estava bem próximo de 41.2.

Note que para cada chute do valor de $\lambda$, precisamos resolver um sistema linear. Então esse método não é muito eficiente.
Existem outros métodos para resolver o problema quadrático de região de confiança, sendo que os mais famosos são o de Dogleg e o de Steihaug.

## Referências

  - [1] J. J. Moré e D. C. Sorensen, _Computing a trust region step_, SIAM Journal on Scientific and Statistical Computing, 4 (1983), pp. 553-572.